## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-29-14-46-16 +0000,nypost,"Trump’s Justice Department probes Google, Veri...",https://nypost.com/2025/12/29/business/trumps-...
1,2025-12-29-14-45-45 +0000,nyt,"For Zelensky, Just Keeping Trump Talking About...",https://www.nytimes.com/2025/12/29/world/europ...
2,2025-12-29-14-41-00 +0000,wsj,SoftBank to Buy Data Center Investment Firm Di...,https://www.wsj.com/business/deals/softbank-gr...
3,2025-12-29-14-38-40 +0000,nypost,"Zelensky splits with Trump, doubts Putin wants...",https://nypost.com/2025/12/29/us-news/volodymy...
4,2025-12-29-14-32-00 +0000,wsj,Ukraine Seeks Decadeslong Security Guarantee a...,https://www.wsj.com/world/europe/ukraine-seeks...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2396/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,29
46,new,16
12,zelensky,13
58,year,13
16,ukraine,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
164,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...,89
9,2025-12-29-13-59-25 +0000,nyt,Trump and Zelensky Meet to Iron Out Peace Plan...,https://www.nytimes.com/2025/12/28/us/politics...,78
1,2025-12-29-14-45-45 +0000,nyt,"For Zelensky, Just Keeping Trump Talking About...",https://www.nytimes.com/2025/12/29/world/europ...,76
156,2025-12-28-20-02-25 +0000,nypost,Putin’s top negotiator posts video of himself ...,https://nypost.com/2025/12/28/world-news/putin...,75
3,2025-12-29-14-38-40 +0000,nypost,"Zelensky splits with Trump, doubts Putin wants...",https://nypost.com/2025/12/29/us-news/volodymy...,73


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
164,89,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...
6,43,2025-12-29-14-17-02 +0000,nyt,Parts of New York and New England Brace for Ic...,https://www.nytimes.com/2025/12/29/us/new-york...
23,41,2025-12-29-12-51-04 +0000,cbc,U.S. offers Ukraine 15-year security guarantee...,https://www.cbc.ca/news/world/ukraine-russia-k...
42,35,2025-12-29-11-32-53 +0000,nyt,China Will Hold Live-Fire Military Exercises A...,https://www.nytimes.com/2025/12/29/world/asia/...
156,27,2025-12-28-20-02-25 +0000,nypost,Putin’s top negotiator posts video of himself ...,https://nypost.com/2025/12/28/world-news/putin...
77,25,2025-12-29-09-19-07 +0000,nypost,Bondi Beach hero Ahmed al Ahmed gives first in...,https://nypost.com/2025/12/29/world-news/bondi...
17,24,2025-12-29-13-23-29 +0000,startribune,"Travel remains difficult in Twin Cities, south...",https://www.startribune.com/travel-remains-dif...
60,24,2025-12-29-10-30-00 +0000,wsj,A life-sciences job in Boston used to be a sur...,https://www.wsj.com/tech/biotech/ph-d-s-cant-f...
28,23,2025-12-29-12-30-00 +0000,missionlocal,‘Fire sale’: S.F. moving to buy out PG&E — eve...,https://missionlocal.org/2025/12/pge-san-franc...
191,21,2025-12-28-15-00-00 +0000,wsj,Gov. JB Pritzker is backing his lieutenant gov...,https://www.wsj.com/politics/elections/jb-prit...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
